In [20]:
import pandas as pd
import numpy as np
from math import pi
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import os
from datetime import datetime
from fastapi.middleware.cors import CORSMiddleware
import xgboost as xgb

In [77]:
# Preprocessor

# dirname = os.path.dirname(__file__)

def clean_data(data):
    
    # 1. Handle Null Values
    # CNAE (only 482)
    data = data.query('cnae != 0')

    # CodiPostal (Most Frequent)
    data.loc[:, 'codipostal'] = data.loc[:, 'codipostal'].replace('-', (data.loc[:, 'codipostal']).mode()[0])

    # Contracte (Delete few wrongs and then Most Frequent)
    data = data[data['contracte'] != '001']
    data = data[data['contracte'] != '019']
    data = data[data['contracte'] != '999']

    mf_contracte = data['contracte'].mode()[0]
    data['contracte'] = data['contracte'].replace('000', mf_contracte).replace('-', mf_contracte)

    # Grupcoti (Most Frequent)
    mf_grupcoti = data['grupcoti'].mode()[0]
    data['grupcoti'] = data['grupcoti'].replace('-', mf_grupcoti).replace('0', mf_grupcoti)

    # 2. Data Cleaning
    # ContadorBajasCCC (non-sense)
    data.drop('ContadorBajasCCC', axis = 1, inplace = True)

    # ContadorBajasDNI vs ContadordiasBajasDNI
    data = data.query('ContadorBajasDNI <= ContadordiasBajasDNI')
    
    return data

def outliers(data):
    
    # Remove some outliers

    data.drop(data[(data['ContadorBajasDNI']>25)].index, axis = 0, inplace = True)
    data.drop(data[(data['numtreb']>3200)].index, axis = 0, inplace = True)
    data.drop(data[(data['ContadordiasBajasDNI']>750)].index, axis = 0, inplace = True)
    
    return data


def categorize(data):
    
    # Categorize CNAE
    cnae_filename = os.path.join('../raw_data/Tabla_cnae.xlsx')
    c = pd.read_excel(cnae_filename, usecols = ['GRUPO', 'COD_CNAE2009'])
    c.rename(columns = {'GRUPO': 'cnae_cat', 'COD_CNAE2009': 'cnae'}, inplace = True)
    c['cnae_cat'] = c['cnae_cat'].map(lambda x: ord(x) - 64)
    c = c.query('cnae.str.isnumeric()').astype('int')
    data = data.merge(c, how = 'left', on = 'cnae')

    # Sickness Types
    block_ends = [140,240,280,290,320,390,460,520,580,630,680,710,740,760,780,800,1000]
    blocks = [[i+1, s] for i, s in enumerate(block_ends)]
    special_cases = ['E','V','M']

    def make_icd9_cat(row):
        first_str = row.icd9[0]
        if first_str == 'E':
            return 18
        if first_str == 'V':
            return 19
        if first_str == 'M':
            return 20
        else:
            code = float(row.icd9)
            for block in blocks:
                if code < block[1]:
                    return block[0]
    data['icd9_cat'] = data.apply(lambda row: make_icd9_cat(row), axis=1)

    # Sickness Wrong Values
    data = data.query('sexo != 1 | icd9_cat != 11')
    data = data[~((data['sexo'] == 1) & (data['icd9'].str[:2].isin(['61', '62'])))]
    data = data[~((data['sexo'] == 2) & (data['icd9'].str[:2] == '60'))]

    # Post Code Categories.
    data['codipostal_cat'] = data['codipostal'].str[:2]

    # Time in company
    # Add 0.01 to have the upper limit. (0.00 means up to 3 days)
    data['proporcion_baja'] = (data['duracion_baja'] / (365 * (data['tiempo_en_empresa'] + 0.01)))
    by_wrong_proportion = data.sort_values(by = 'proporcion_baja', axis = 0, ascending = False)
    data = data.query('proporcion_baja <= 1')
    data.drop(columns = 'proporcion_baja', axis = 1, inplace = True)

    # Month not required
    data.drop(columns = 'mes_baja', axis = 1, inplace = True)

    # Cyclic Calendar
    data.loc[:, 'time'] = (2 * pi * (data.loc[:, 'epiweek'] - 1) + ((data.loc[:, 'diasemana'] - 1) / 7)) / 52
    data.loc[:, 'sin_time'] = np.sin(data.loc[:, 'time'])
    data.loc[:, 'cos_time'] = np.cos(data.loc[:, 'time'])
    data.drop(['time', 'diasemana', 'epiweek'], axis = 1, inplace = True)
    
    return data


def data_for_ml(data):

    # 3. Prepare Data for ML
    data.drop(['cnae', 'icd9', 'codipostal', 'contracte'], axis = 1, inplace = True)

    sample = data.sample(20000, random_state = 8)
    # sample = data
    X_train, X_test, y_train, y_test = train_test_split(
                            sample.drop('duracion_baja', axis = 1), sample['duracion_baja'], test_size=0.3)

    def prepare_data(X_train, X_test):
        # One hot Encode Categorical Features.
        ohe_cols = ['sexo', 'recaida', 'grupcoti', 'pluriempleo',
                    'cnae_cat', 'icd9_cat', 'codipostal_cat']
        for col in ohe_cols:
            ohe = OneHotEncoder(sparse = False, handle_unknown = 'ignore')
            ohe.fit(X_train[[col]])
            for X_t in [X_train, X_test]:
                X_t[ohe.get_feature_names_out()] = ohe.transform(X_t[[col]])
                X_t.drop(col, axis = 1, inplace = True)


        # Scale numerical features.
        robust_cols = ['ContadorBajasDNI', 'ContadordiasBajasDNI', 'numtreb', 'tiempo_en_empresa']
        for col in robust_cols:
            rb_scaler = RobustScaler()
            rb_scaler.fit(X_train[[col]])
            for X_t in [X_train, X_test]:
                X_t[col] = rb_scaler.transform(X_t[[col]])

        st_scaler = StandardScaler()
        st_scaler.fit(X_train[['edad']])
        for X_t in [X_train, X_test]:
            X_t['edad'] = st_scaler.transform(X_t[['edad']])

        return X_train, X_test, ohe, rb_scaler, st_scaler

    X_train_p, X_test_p, ohe, rb_scaler, st_scaler = prepare_data(X_train, X_test)
    
    return X_train_p, X_test_p, y_train, y_test, ohe, rb_scaler, st_scaler


def prepare_pred(X_new, ohe, rb_scaler, st_scaler):
    # 3. Prepare Data for ML
    X_new.drop(['cnae', 'icd9', 'codipostal', 'contracte'], axis = 1, inplace = True)
    # One hot Encode Categorical Features.
    ohe_cols = ['sexo', 'recaida', 'grupcoti', 'pluriempleo',
                'cnae_cat', 'icd9_cat', 'codipostal_cat']
    for col in ohe_cols:
        for X_t in [X_new]:
            X_t[ohe.get_feature_names_out()] = ohe.transform(X_t[[col]])
            X_t.drop(col, axis = 1, inplace = True)
            
    # Scale numerical features.
    robust_cols = ['ContadorBajasDNI', 'ContadordiasBajasDNI', 'numtreb', 'tiempo_en_empresa']
    for col in robust_cols:
        for X_t in [X_new]:
            X_t[col] = rb_scaler.transform(X_t[[col]])
        
    for X_t in [X_new]:
        X_t['edad'] = st_scaler.transform(X_t[['edad']])
        
    return X_new
    
    
def preprocess_train(data):
    data = clean_data(data)
    data = outliers(data)
    data = categorize(data)
    X_train, X_test, y_train, y_test, ohe, rb_scaler, st_scaler = data_for_ml(data)
    
    return X_train, X_test, y_train, y_test, ohe, rb_scaler, st_scaler


def preprocess_new(data, ohe, rb_scaler, st_scaler):
    data.drop('ContadorBajasCCC', axis = 1, inplace = True)
    data = categorize(data)
    X_new_prepr = prepare_pred(data, ohe, rb_scaler, st_scaler)
    
    return X_new_prepr

In [28]:
# Model

def fit_model(X_train, y_train):
    model = xgb.XGBRegressor(booster= 'dart', objective ='reg:squarederror', n_estimators = 750, learning_rate= 0.01,
                               min_child_weight= 4, max_depth= 8, colsample_bytree= 0.7, n_jobs=-1)
    y_log_train = np.log(y_train)
    model.fit(X_train, y_log_train)
    
    return model

In [29]:
# api

# dirname = os.path.dirname(__file__)

def load_data():
    
    # 0. Load Data and Drop Duplicates
    # csv_filename = os.path.join(dirname, '../raw_data/datos.csv')
    data = pd.read_csv('../raw_data/datos.csv', delimiter = ';', low_memory = False)
    data.drop_duplicates(inplace = True)
    # outliers = data.query('duracion_baja >= 250')  # Save outliers for analysis.
    data = data.query('duracion_baja < 250')
    # data.drop_duplicates(inplace = True)
    
    return data



In [31]:
data = load_data()
X_train, X_test, y_train, y_test, ohe, rb_scaler, st_scaler = preprocess_train(data)
model = fit_model(X_train, y_train)

/var/folders/jt/ljcpcwg95nv566n6vzs4822m0000gn/T/ipykernel_4737/1900875715.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'codipostal'] = data.loc[:, 'codipostal'].replace('-', (data.loc[:, 'codipostal']).mode()[0])
/Users/jaimesoroarubio/.pyenv/versions/3.10.6/envs/smarthealing/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/var/folders/jt/ljcpcwg95nv566n6vzs4822m0000gn/T/ipykernel_4737/1900875715.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [78]:
data

,ContadorBajasCCC,ContadorBajasDNI,sexo,cnae,icd9,recaida,numtreb,codipostal,ContadordiasBajasDNI,contracte,grupcoti,pluriempleo,diasemana,tiempo_en_empresa,edad,duracion_baja,mes_baja,epiweek
0,17664,30,2,5110,009.3,0,3187,08037,224,420,2,0,7,0.51,27.2,2,12,51
1,17664,5,2,5110,787.01,0,3187,39006,12,420,2,0,4,1.41,31.8,1,12,52
2,17664,17,2,5110,462,0,3187,38360,46,420,2,0,4,0.54,20.5,1,12,52
3,17664,16,2,5110,460,0,3187,08830,49,100,2,0,4,2.53,30.6,3,12,52
4,17664,7,1,5110,462,0,3187,02001,548,189,1,0,5,6.16,32.4,2,12,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257755,41,9,2,8121,784.0,0,8,08020,21,200,10,0,7,1.99,44.9,1,1,53
257756,36,13,2,9319,525.8,0,0,08901,223,289,7,1,7,9.68,45.1,1,1,53
257757,9,13,2,5630,525.8,0,0,08901,223,200,8,1,7,3.75,45.1,1,1,53
257758,1,1,2,9700,053.9,0,1,08028,1,200,0,0,7,4.26,36.5,1,1,53


In [80]:
X_new = data[0:1]

In [81]:
X_new

,ContadorBajasCCC,ContadorBajasDNI,sexo,cnae,icd9,recaida,numtreb,codipostal,ContadordiasBajasDNI,contracte,grupcoti,pluriempleo,diasemana,tiempo_en_empresa,edad,duracion_baja,mes_baja,epiweek
0,17664,30,2,5110,009.3,0,3187,08037,224,420,2,0,7,0.51,27.2,2,12,51


In [82]:
X_new_prepr = preprocess_new(X_new, ohe, rb_scaler, st_scaler)    


/var/folders/jt/ljcpcwg95nv566n6vzs4822m0000gn/T/ipykernel_4737/3865465782.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop('ContadorBajasCCC', axis = 1, inplace = True)
/Users/jaimesoroarubio/.pyenv/versions/3.10.6/envs/smarthealing/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/Users/jaimesoroarubio/.pyenv/versions/3.10.6/envs/smarthealing/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- sexo
Feature names seen at fit time, yet now missing:
- codipostal_cat

  warnings.war

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [76]:
y_pred = model.predict(X_new_prepr)

ValueError: Feature shape mismatch, expected: 113, got 19